In [2]:
import pandas as pd
import numpy as np
import joblib
import os
import json
import warnings
import sklearn.utils.validation

warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
# [변경 1] 일반 SMOTE 대신 경계선 데이터를 집중 공략하는 BorderlineSMOTE 사용
from imblearn.over_sampling import BorderlineSMOTE 
from imblearn.pipeline import Pipeline

# 데이터 로드
print("데이터 불러오는 중...")
df = pd.read_csv('data/spotify_churn_dataset.csv')

# 불필요한 ID 컬럼 제거
if 'user_id' in df.columns:
    df = df.drop(columns=['user_id'])

# Feature Engineering (파생변수 추가)
df['ad_burden'] = df['ads_listened_per_week'] / (df['listening_time'] + 1)
df['satisfaction_score'] = df['songs_played_per_day'] * (1 - df['skip_rate'])
df['time_per_song'] = df['listening_time'] / (df['songs_played_per_day'] + 1)

# X, y 분리
X = df.drop(columns=['is_churned'])
y = df['is_churned']

# 컬럼 분류
numerical_cols = [
    'age', 'listening_time', 'songs_played_per_day', 'skip_rate', 
    'ads_listened_per_week', 'offline_listening',
    'ad_burden', 'satisfaction_score', 'time_per_song'
]
categorical_cols = ['gender', 'country', 'subscription_type', 'device_type']

# 전처리기
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
    ])

# 학습 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# [변경 2] 모델별 가중치(Weights) 강화 - F1 점수를 위해 이탈자(1)를 더 중요하게 취급
models_params = {
    "RandomForest": {
        "model": RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced_subsample'), # subsample로 변경하여 더 정교하게
        "params": {
            "classifier__n_estimators": [500, 1000],
            "classifier__max_depth": [20, None]
        }
    },
    "XGBoost": {
        # scale_pos_weight: 이탈자(Positive)에 가중치 2배 부여
        "model": XGBClassifier(random_state=42, n_jobs=-1, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=2.0),
        "params": {
            "classifier__n_estimators": [1000],
            "classifier__learning_rate": [0.05, 0.1],
            "classifier__max_depth": [5, 7]
        }
    },
    "LightGBM": {
        "model": LGBMClassifier(random_state=42, n_jobs=-1, verbose=-1, force_col_wise=True, scale_pos_weight=2.0),
        "params": {
            "classifier__n_estimators": [1000],
            "classifier__learning_rate": [0.05, 0.1]
        }
    },
    "CatBoost": {
        # SqrtBalanced: F1 Score 최적화에 유리한 가중치 옵션
        "model": CatBoostClassifier(random_state=42, verbose=0, allow_writing_files=False, auto_class_weights='SqrtBalanced'),
        "params": {
            "classifier__iterations": [1000],
            "classifier__learning_rate": [0.05, 0.1]
        }
    }
}

best_estimators = []
metrics_data = {}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# BorderlineSMOTE 설정
bsmote = BorderlineSMOTE(random_state=42, kind='borderline-1')

print("모델 학습 및 최적화 시작...")

for name, config in models_params.items():
    print(f"{name} 최적화 중...")
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', bsmote), # 일반 SMOTE -> BorderlineSMOTE 교체
        ('classifier', config['model'])
    ])
    
    search = RandomizedSearchCV(
        pipeline, config['params'], n_iter=10, scoring='f1', cv=cv, n_jobs=-1, random_state=42, verbose=0
    )
    search.fit(X_train, y_train)
    
    # 평가
    best_model = search.best_estimator_
    y_pred = best_model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    metrics_data[name] = {'Accuracy': acc, 'F1-Score': f1}
    print(f"  - {name} Accuracy: {acc:.4f}, F1: {f1:.4f}")
    
    # 앙상블을 위한 파라미터 추출 및 모델 재생성
    best_params_clean = {k.replace('classifier__', ''): v for k, v in search.best_params_.items()}
    
    if name == "RandomForest":
        model_instance = RandomForestClassifier(**best_params_clean, random_state=42, n_jobs=-1, class_weight='balanced_subsample')
    elif name == "XGBoost":
        model_instance = XGBClassifier(**best_params_clean, random_state=42, n_jobs=-1, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=2.0)
    elif name == "LightGBM":
        model_instance = LGBMClassifier(**best_params_clean, random_state=42, n_jobs=-1, verbose=-1, force_col_wise=True, scale_pos_weight=2.0)
    elif name == "CatBoost":
        model_instance = CatBoostClassifier(**best_params_clean, random_state=42, verbose=0, allow_writing_files=False, auto_class_weights='SqrtBalanced')
        
    best_estimators.append((name, model_instance))

# 최종 Voting 앙상블
print("최종 앙상블 모델 학습 중...")
voting_clf = VotingClassifier(estimators=best_estimators, voting='soft')

final_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('smote', bsmote), # 최종 단계에서도 BorderlineSMOTE 사용
    ('classifier', voting_clf)
])

final_pipeline.fit(X_train, y_train)

# [변경 3] 최적의 임계값(Threshold) 찾기 - F1 Score 극대화의 핵심
print("최적의 임계값 탐색 중...")
y_pred_proba = final_pipeline.predict_proba(X_test)[:, 1] # 이탈 확률만 추출

thresholds = np.arange(0.3, 0.7, 0.01) # 0.3 ~ 0.7 사이를 0.01 단위로 검사
best_thresh = 0.5
best_f1_val = 0
best_acc_val = 0

for thresh in thresholds:
    y_pred_temp = (y_pred_proba >= thresh).astype(int)
    f1_temp = f1_score(y_test, y_pred_temp)
    
    if f1_temp > best_f1_val:
        best_f1_val = f1_temp
        best_acc_val = accuracy_score(y_test, y_pred_temp)
        best_thresh = thresh

# 최종 결과 적용
final_acc = best_acc_val
final_f1 = best_f1_val

print("-" * 40)
print(f"🚀 최적 임계값 발견: {best_thresh:.2f}")
print(f"최종 Accuracy: {final_acc:.4f}")
print(f"최종 F1-Score: {final_f1:.4f}")
print("-" * 40)

# 저장
if not os.path.exists('models'): os.makedirs('models')
# 주의: pipeline 자체는 predict_proba를 뱉으므로, 나중에 불러올 때 threshold 로직을 따로 써야 함
# 여기서는 모델 객체만 저장
joblib.dump(final_pipeline, 'models/spotify_churn_model.pkl')

# 메트릭 저장 (JSON)
metrics_file = 'data/model_metrics.json'
final_metrics = {}
if os.path.exists(metrics_file):
    try:
        with open(metrics_file, 'r') as f:
            final_metrics = json.load(f)
    except:
        pass

# 메트릭에 임계값 정보도 같이 저장하면 좋음
metrics_data['Voting Ensemble (ML)'] = {
    'Accuracy': final_acc, 
    'F1-Score': final_f1,
    'Best Threshold': float(best_thresh)
}
final_metrics.update(metrics_data)

with open(metrics_file, 'w') as f:
    json.dump(final_metrics, f, indent=4)

print("모델 및 점수 저장 완료.")

데이터 불러오는 중...
모델 학습 및 최적화 시작...
RandomForest 최적화 중...
  - RandomForest Accuracy: 0.9445, F1: 0.8871
XGBoost 최적화 중...
  - XGBoost Accuracy: 0.9460, F1: 0.8861
LightGBM 최적화 중...
  - LightGBM Accuracy: 0.9485, F1: 0.8896
CatBoost 최적화 중...
  - CatBoost Accuracy: 0.9525, F1: 0.8973
최종 앙상블 모델 학습 중...
최적의 임계값 탐색 중...
----------------------------------------
🚀 최적 임계값 발견: 0.34
최종 Accuracy: 0.9500
최종 F1-Score: 0.8971
----------------------------------------
모델 및 점수 저장 완료.
